In [ ]:
"""
Import modules
"""
import os, time
import warnings
import pickle
import cv2
import numpy as np
import torch

from utils import load_model, preprocess_image, visualize
from Explanation import CAM_Explanation

In [ ]:
"""
Required parameters
    - gpu_id : gpu device number.
    - model_archi : original prediction model; 1)"vgg16" or 2)"resnet50" are possible options.
    - ckpt_path : model's checkpoint path. If None, you will use an off-the-shelf model pretrained on ImageNet.
    - img_path : the path of sample image which you want to analyze.
    - method : saliency method; "LIFT-CAM", "LRP-CAM", and "LIME-CAM" are the possible options.
"""
gpu_id = "0"
model_archi = "vgg16"
ckpt_path = None
img_path = "./notebook.jpg" 
method = "LIFT-CAM"

In [ ]:
"""
Preparation
"""
warnings.filterwarnings("ignore")
os.environ["CUDA_VISIBLE_DEVICES"] = gpu_id
idx_to_label = pickle.load(open("./idx2label.p","rb"))

In [ ]:
"""
Prediction
"""
original_img = cv2.imread(img_path, 1)
original_img_resized = cv2.resize(original_img,(224,224))
img = preprocess_image(np.float32(original_img_resized)/255)
model = load_model(model_archi, ckpt_path)
output = torch.nn.Softmax()(model(img))
category = int(torch.argmax(output).cpu().detach().numpy())
print("Predicted Label: {}".format(idx_to_label[category]))

In [ ]:
"""
Attribution
"""
CAM = CAM_Explanation(model, method)
start = time.time()
vis_ex_map = CAM(img.cpu(),int(category),(224,224))
end = time.time()
print("Elapsed time: {} sec".format(end-start))

In [ ]:
"""
Visualization
"""
visualize(original_img_resized, vis_ex_map, method)